In [ ]:
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma
from valentine.algorithms import Cupid
import os
import multiprocessing as mp
from multiprocessing import Pool
import time
import pandas as pd
from compute_match import *

# Preparazione data frames

In [ ]:
csv_path = './csv/'

## Creazione dizionari che associa il nome del file al file .json

In [ ]:
# dataset sono in formato json necessariamente
path = "../../json_datasets"

# Dizionario Key=Nome del file, Value=File json associato (letto da filepath)
data_frames = {}
# Per tutti i file dentro la cartella dei dataset
for file in os.listdir(path):
    # Prendo il path del file
    filepath = f"{path}/{file}"
    # Update del dizionario
    data_frames[file] = (pd.read_json(filepath))

## Creazione di tutte le coppie di dataset da valutare (evitando ripetizioni)

In [ ]:
# Prendo tutti i nomi dei file
keys = list(data_frames.keys())
tuples = []
# Il numero di dataframes
dfs_len = len(keys)

In [ ]:
# Preso un dataframe, lo confronto con tutti i dataframes successivi (così evito di confrontare coppie già confrontate)
for i in range(dfs_len):
    for j in range(i + 1, dfs_len):
        dfl = data_frames[keys[i]]
        dfr = data_frames[keys[j]]
        # le tuple da passare al metodo di matching = nome dataset1, nome dataset2, dataframe1, dataframe2
        tuples.append((keys[i], keys[j], dfl, dfr))

# Valentine schema matching

In [ ]:
def calculate_match_multithread(tuples):
    result = []
    # Creo un pool per l'esecuzione multi processore (per velocizzare il matching tra tutte le coppie di dataframes)
    with Pool() as pool:
        # Ogni pool esegue la funzione (primo parametro), passandogli le tuple (secondo parametro)
        # result è una lista di tuple ove una tupla è il risultato della funzione mappata
        result = pool.map(calculate_match_coma_schema, tuples)
    return result

def matches_to_table(result):
    matches = pd.DataFrame()
    # Per ogni risultato ottenuto dal matching
    for match in result:
    # Prendo il valore ritornato dalla funzione valentine_match 
    # e.g. match[2] = ((table_1, 'Cited by'),(table_2, 'Cited by')): 0.83
        for key in match[2].keys():
            # e.g. key = (table_1, 'Cited by'),(table_2, 'Cited by')
            d = dict()
            d["table_1"] = match[0]
            d["table_2"] = match[1]
            d["column_table_1"] = key[0][1]
            d["column_table_2"] = key[1][1]
            d["match_value"] = match[2][key]
            # Aggiungiamo al dataframe l'ennupla
            matches = pd.concat([matches, pd.DataFrame(d, index=[0])], ignore_index=True)
    return matches

# Tabella per la visualizzazione dei risultati dello schema matching con Valentine

In [ ]:
start_time = time.perf_counter()
result = calculate_match_multithread(tuples)
finish_time = time.perf_counter()

# Definizione del nome delle colonne della tabella
columns = ["table_1", "table_2", "column_table_1", "column_table_2", "match_value"]
# Creo il dataframe (la tabella in questione)
matches = pd.DataFrame(columns=columns)

matches = matches_to_table(result)
print(matches)

In [ ]:
pd.DataFrame(matches).to_csv(csv_path + "matches.csv")

# Schema mediato

In [ ]:
mediated_schema_columns = ('name', 'country', 'market cap', 'founded year', 'employees', 'industry', 'sector',
     'ceo', 'revenue', 'Stock', 'share price', 'city', 'address', 'website')

mediated_schema = pd.DataFrame(columns=mediated_schema_columns)

In [ ]:
dfs_len = len(keys)
matches_with_mediated = {}
for i in range(dfs_len):
    dfl = mediated_schema
    dfr = data_frames[keys[i]]
    # le tuple da passare al metodo di matching = nome dataset1, nome dataset2, dataframe1, dataframe2
    tuples.append(('mediated_schema', keys[i], dfl, dfr))

In [ ]:
result = calculate_match_multithread(tuples)

In [ ]:
matches = matches_to_table(result)
print(matches)

In [ ]:
pd.DataFrame(matches).to_csv(csv_path + "matches_with_mediated.csv")